# Results Notebook

In [1]:
import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
df = pd.DataFrame
sns.set_theme()

def melt_row_col_data(arr,name):
    arr_df = df(arr)
    arr_df = arr_df.melt(
        ignore_index=False,
        var_name='Column',
        value_name='ADC_OUT_AMS',
    )
    arr_df['Row'] = arr_df.index
    return arr_df

In [2]:
# Generate the reference result

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)
wx = w @ x.T
wx_rounded = np.round(wx.T / 2**2).astype(int).T[::-1].T

[STIM_GEN] Generating random weights


### Comparing AMS vs RTL simulations of QRAcc

In [3]:
ams = np.loadtxt('../tb/qracc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')

ams_df = df(ams)
rtl_df = df(rtl)

# ams_df['Row'] = ams_df.index
# rtl_df['Row'] = rtl_df.index

ams_df = ams_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_AMS',
)
rtl_df = rtl_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_RTL',
)
ams_df['Row'] = ams_df.index
rtl_df['Row'] = rtl_df.index
# Join the two dataframes
adc_df = pd.merge(ams_df, rtl_df)

rmse = np.sqrt(np.mean((rtl.flatten() - ams.flatten())**2))
fig = px.scatter(adc_df, y='ADC_OUT_AMS', x='ADC_OUT_RTL', width=400, height=400, title=f'AMS vs Shifted, RMSE={rmse:.2f}', labels={'ADC_OUT_AMS':'AMS','ADC_OUT_RTL':'RTL'})
fig.show()
fig.write_image('../images/rtl_vs_ams.svg')

FileNotFoundError: ../tb/qracc/inputs/adc_out.txt not found.

In [17]:
px.scatter(x=wx_rounded.flatten(),y=rtl.flatten(),title='Rounded vs Shifted',labels={'x':'Rounded','y':'Shifted'}, width=400, height=400)

In [18]:
rmse = np.sqrt(np.mean((ams.flatten() - wx_rounded.flatten())**2))
px.scatter(x=wx_rounded.flatten(),y=ams.flatten(),labels={'x':'Rounded','y':'AMS'}, width=400, height=400, title=f'AMS vs Rounded, RMSE={rmse:.2f}')

In [19]:
# pd.concat([df(ams),df(wx_rounded)],names=['ams','rtl']).T
df(np.array([str(i) for i in zip(ams.flatten().astype(int),wx_rounded.flatten())]).reshape(10,-1)).T
# np.array([str(i) for i in zip(ams.flatten(),wx_rounded.flatten())]).reshape(10,-1,2)

,0,1,2,3,4,5,6,7,8,9
0,"(6, 6)","(0, 0)","(0, 0)","(-1, -2)","(1, 0)","(0, 0)","(2, 2)","(4, 3)","(0, 0)","(-2, -2)"
1,"(-2, -2)","(3, 3)","(-5, -5)","(0, -1)","(0, 0)","(1, 1)","(-2, -2)","(1, 0)","(3, 3)","(0, 0)"
2,"(4, 4)","(3, 3)","(-1, -1)","(-3, -3)","(-3, -4)","(1, 1)","(2, 2)","(-1, -1)","(2, 2)","(-3, -3)"
3,"(2, 1)","(0, -1)","(-2, -2)","(3, 2)","(2, 2)","(-4, -4)","(0, 0)","(2, 1)","(2, 2)","(0, 0)"
4,"(1, 1)","(-1, -1)","(-1, -1)","(1, 0)","(-2, -3)","(6, 6)","(3, 3)","(-1, -2)","(-1, -2)","(1, 0)"
5,"(1, 0)","(0, 0)","(0, -1)","(1, 1)","(2, 2)","(2, 2)","(-5, -6)","(0, -1)","(1, 0)","(1, 0)"
6,"(0, 0)","(-2, -2)","(2, 1)","(-2, -3)","(1, 1)","(2, 2)","(0, 0)","(0, -1)","(2, 2)","(1, 1)"
7,"(0, -1)","(2, 2)","(2, 2)","(3, 2)","(6, 6)","(-1, -2)","(1, 1)","(-1, -1)","(-2, -2)","(4, 4)"
8,"(3, 2)","(-1, -1)","(0, -1)","(1, 1)","(2, 2)","(3, 2)","(1, 1)","(3, 2)","(5, 4)","(1, 0)"
9,"(2, 2)","(-1, -2)","(-4, -4)","(1, 1)","(1, 0)","(0, 0)","(-1, -2)","(-1, -1)","(4, 4)","(2, 2)"


In [20]:
pd.concat([df(ams)-df(wx_rounded)],names=['ams','rtl']).T

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
9,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [21]:
# pd.concat([df(ams),df(wx_rounded)],names=['ams','rtl']).T
df(np.array([str(i) for i in zip(ams.flatten().astype(int),rtl.flatten().astype(int))]).reshape(10,-1)).T
# np.array([str(i) for i in zip(ams.flatten(),wx_rounded.flatten())]).reshape(10,-1,2)

,0,1,2,3,4,5,6,7,8,9
0,"(6, 5)","(0, 0)","(0, -1)","(-1, -2)","(1, 0)","(0, -1)","(2, 1)","(4, 3)","(0, 0)","(-2, -3)"
1,"(-2, -3)","(3, 3)","(-5, -6)","(0, -1)","(0, 0)","(1, 1)","(-2, -3)","(1, 0)","(3, 3)","(0, -1)"
2,"(4, 4)","(3, 2)","(-1, -2)","(-3, -4)","(-3, -4)","(1, 1)","(2, 2)","(-1, -2)","(2, 1)","(-3, -3)"
3,"(2, 1)","(0, -1)","(-2, -3)","(3, 2)","(2, 1)","(-4, -4)","(0, 0)","(2, 1)","(2, 1)","(0, -1)"
4,"(1, 0)","(-1, -2)","(-1, -2)","(1, 0)","(-2, -3)","(6, 6)","(3, 3)","(-1, -2)","(-1, -2)","(1, 0)"
5,"(1, 0)","(0, -1)","(0, -1)","(1, 0)","(2, 1)","(2, 1)","(-5, -6)","(0, -1)","(1, 0)","(1, 0)"
6,"(0, -1)","(-2, -3)","(2, 1)","(-2, -3)","(1, 1)","(2, 2)","(0, 0)","(0, -1)","(2, 1)","(1, 1)"
7,"(0, -1)","(2, 2)","(2, 1)","(3, 2)","(6, 5)","(-1, -2)","(1, 1)","(-1, -2)","(-2, -3)","(4, 3)"
8,"(3, 2)","(-1, -2)","(0, -1)","(1, 0)","(2, 1)","(3, 2)","(1, 1)","(3, 2)","(5, 4)","(1, 0)"
9,"(2, 1)","(-1, -2)","(-4, -5)","(1, 0)","(1, 0)","(0, -1)","(-1, -2)","(-1, -2)","(4, 3)","(2, 1)"


### Save current dataFrame

In [22]:
adc_df.to_csv('tb_qracc_results.csv', index=False,)